# Econometric Feature Creation

In [1]:
#imports 
import pandas as pd
import numpy as np
import os
import statsmodels.formula.api as sm
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange, tqdm
import statsmodels.api as sm
import networkx as nx
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold

In [2]:
import pandas as pd
import requests

def get_sitc_codes():
    # URL of the JSON file
    url = 'https://comtradeapi.un.org/files/v1/app/reference/S4.json'

    try:
        # Send a GET request to the URL and fetch the data
        response = requests.get(url)
        response.raise_for_status()  # Check that the request was successful
        
        # Load the JSON data
        data = response.json()

        # Since the JSON data might be nested, use json_normalize with appropriate arguments
        if isinstance(data, list):
            # If the top level is a list
            df = pd.json_normalize(data)
        else:
            # If the top level is a dictionary
            # Identify the key that holds the main data (adjust the path as necessary)
            main_data_key = 'results'  # Adjust this based on the actual structure
            df = pd.json_normalize(data[main_data_key])

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
    except KeyError as e:
        print(f"Error processing JSON structure: {e}")

    return df

In [3]:
#we mimic 4-class system from World Bank -- 4 growth categoris: 0 - L, 1 - LM, 2 - UM, 3 - U
def growth_categories(x, growth_list):
    threshold_high = np.percentile(growth_list, 75)
    threshold_low = np.percentile(growth_list, 25)
    threshold_mid = np.percentile(growth_list, 50)
    
    if(x > threshold_high): return 3
    elif(x < threshold_low): return 0
    elif(x < threshold_mid): return 1
    else: return 2

In [4]:
class CreateFeatures:
    """
    We define a class which builds the feature dataframe 
    """
    
    def __init__(self, year = 1962, data_dir = "../data/"):
        self.year = year
        self.data_dir = data_dir
        
    def prepare_econ_features(self, filter_gdp = True):
        
        #DATA IMPORT
        #import dictionary with all features from WB
        with open(self.data_dir + 'all_wb_indicators.pickle', 'rb') as handle:
            features_dict = pkl.load(handle)
            
        self.feature_list = list(features_dict.keys())[1:]
        #import list of all features we want to select for

        #look up each of the features -- add country feature in that year 
        i = 0
        for feature in self.feature_list:
            #find dataframe corresponding to specific feature name
            df = features_dict[feature]
            
            if (i == 0):
                self.features = df[["economy", "YR" + str(self.year)]]
            else: 
                self.features = pd.merge(self.features, 
                                            df[["economy", "YR" + str(self.year)]],
                                            on = "economy", how = "outer")
            self.features.rename(columns = {"YR" + str(self.year): feature}, inplace = True)
            i = i+1
        
        #prepare GDP feature
        self.gdp_growth = features_dict['NY.GDP.MKTP.KD.ZG']
        cols = list(self.gdp_growth.columns.copy())
        cols.remove("economy")
        self.gdp_growth["country_sd"] = self.gdp_growth[cols].std(axis=1)
        #select potential variables 
        self.gdp_growth["prev_gdp_growth"] = self.gdp_growth["YR" + str(self.year-1)]
        self.gdp_growth["current_gdp_growth"] = self.gdp_growth["YR" + str(self.year)]
        #we eliminate countries that are too volatile in growth -- probably an indicator that growth estimates are inaccurate
        self.gdp_growth = self.gdp_growth[["economy", "prev_gdp_growth",
                                "current_gdp_growth"]].dropna()
        
        #combine GDP and other features
        self.features = pd.merge(self.gdp_growth, self.features,
                                   on = "economy", how = "left")
        #we only keep countries where we observe GDP growth -- otherwise nothing to predict
        #we keep countries where other features may be missing -- and fill NAs with 0 
        self.features.rename(columns = {"economy": "country_code"}, inplace = True)
        
    def prepare_network_features(self):
        """
        We create an initial, import-centric trade link pandas dataframe for a given year
        """
        #get product codes
        data_dict = get_sitc_codes()
        data_cross = []
        i = 0
        for item_def in list(data_dict["text"]):
            if(i >= 2):
                data_cross.append(item_def.split(" - ", 1))
            i = i+1

        self.product_codes = pd.DataFrame(data_cross, columns = ['code', 'product'])
        self.product_codes["sitc_product_code"] = self.product_codes["code"]
        
        #get country codes
        self.country_codes = pd.read_excel(self.data_dir + "ISO3166.xlsx")
        self.country_codes["location_code"] = self.country_codes["Alpha-3 code"]
        self.country_codes["partner_code"] = self.country_codes["Alpha-3 code"]
        self.country_codes["country_i"] = self.country_codes["English short name"]
        self.country_codes["country_j"] = self.country_codes["English short name"]
        
        #get trade data for a given year
        trade_data = pd.read_stata(self.data_dir + "country_partner_sitcproduct4digit_year_"+ str(self.year)+".dta") 
        #merge with product / country descriptions
        trade_data = pd.merge(trade_data, self.country_codes[["location_code", "country_i"]],on = ["location_code"])
        trade_data = pd.merge(trade_data, self.country_codes[["partner_code", "country_j"]],on = ["partner_code"])
        trade_data = pd.merge(trade_data, self.product_codes[["sitc_product_code", "product"]], 
                              on = ["sitc_product_code"])
        ###select level of product aggregation
        trade_data["product_category"] = trade_data["sitc_product_code"].apply(lambda x: x[0:1])
        #trade_data = trade_data[trade_data["product_category"] == "1"]
        
        #keep only nodes that we have features for
        trade_data = trade_data[trade_data["location_code"].isin(self.features["country_code"])]
        trade_data = trade_data[trade_data["partner_code"].isin(self.features["country_code"])]
        
        if (len(trade_data.groupby(["location_code", "partner_code", "sitc_product_code"])["import_value"].sum().reset_index()) != len(trade_data)):
            print("import, export, product combination not unique!")
        self.trade_data1 = trade_data
        #from import-export table, create only import table
        #extract imports
        imports1 = trade_data[['location_id', 'partner_id', 'product_id', 'year',
               'import_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        imports1 = imports1[imports1["import_value"] != 0]
        #transform records of exports into imports
        imports2 = trade_data[['location_id', 'partner_id', 'product_id', 'year',
               'export_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        imports2["temp1"] = imports2['partner_code']
        imports2["temp2"] = imports2['location_code']

        imports2['location_code'] = imports2["temp1"]
        imports2['partner_code'] = imports2["temp2"]
        imports2["import_value"] = imports2["export_value"]
        imports2 = imports2[imports2["import_value"] != 0]
        imports2 = imports2[['location_id', 'partner_id', 'product_id', 'year',
               'import_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        
        imports_table = pd.concat([imports1, imports2]).drop_duplicates()
        
        #rename columns for better clarity
        imports_table["importer_code"] = imports_table["location_code"]
        imports_table["exporter_code"] = imports_table["partner_code"]
        imports_table["importer_name"] = imports_table["country_i"]
        imports_table["exporter_name"] = imports_table["country_j"]
        
        cols = ["importer_code", "exporter_code", "importer_name", "exporter_name",
               'product_id', 'year', 'import_value', 'sitc_eci', 'sitc_coi',
               'sitc_product_code', 'product', "product_category"]
        imports_table = imports_table[cols]
        
        exporter_total = imports_table.groupby(["exporter_code"])["import_value"].sum().reset_index()
        exporter_total = exporter_total.rename(columns = {"import_value": "export_total"})
        
        importer_total = imports_table.groupby(["importer_code"])["import_value"].sum().reset_index()
        importer_total = importer_total.rename(columns = {"import_value": "import_total"})
        
        ##### COMPUTE CENTRALITY FOR COUNTRY
        #sum imports across all products between countries into single value 
        imports_table_grouped = imports_table.groupby(["importer_code", "exporter_code"])["import_value"].sum().reset_index()
        imports_table_grouped = pd.merge(imports_table_grouped, importer_total, on = "importer_code")
        imports_table_grouped["import_fraction"] = imports_table_grouped["import_value"]\
                        /imports_table_grouped["import_total"]*100
        
        self.trade_data = imports_table_grouped
        
        #filter features and nodes to ones that are connected to others in trade data
        list_active_countries = list(set(list(self.trade_data ["importer_code"])+\
                        list(self.trade_data ["exporter_code"])))
        self.features = self.features[self.features["country_code"].isin(list_active_countries)].reset_index()
        self.features["node_numbers"] = self.features.index
        
        G=nx.from_pandas_edgelist(self.trade_data, 
                          "exporter_code", "importer_code", create_using = nx.DiGraph())
        
        self.G = G
        self.centrality_overall= nx.eigenvector_centrality(G, max_iter= 10000) 
        self.centrality_overall = pd.DataFrame(list(map(list, self.centrality_overall.items())), 
                                               columns = ["country_code", "centrality_overall"])
        G=nx.from_pandas_edgelist(self.trade_data, 
                          "exporter_code", "importer_code", ["import_fraction"])
        weighted_centrality = nx.eigenvector_centrality(G, weight = "import_fraction", max_iter= 10000) 
        weighted_centrality  = pd.DataFrame(list(map(list, weighted_centrality.items())), 
                                               columns = ["country_code", "weighted_centrality"])
        self.centrality_overall = pd.merge(self.centrality_overall, weighted_centrality, on = "country_code")
        
                               
        ##### COMPUTE CENTRALITY FOR COUNTRY IN PRODUCT CATEGORIES

        #sum imports across all products between countries into single value 
        imports_table_grouped = imports_table.groupby(["importer_code", "exporter_code"])["import_value"].sum().reset_index()
        products_grouped = imports_table.groupby(["product_category"])["import_value"].sum().reset_index()
        products_grouped = products_grouped.rename(columns = {"import_value": "import_product_total"})
        
        #sum exports in each category 
        self.export_types = imports_table.groupby(["importer_code", "exporter_code", "product_category"])["import_value"].sum().reset_index()
        self.export_types = pd.merge(products_grouped, self.export_types, on = "product_category")
        
        self.export_types["product_export_fraction"] = self.export_types["import_value"]\
                                                    /self.export_types["import_product_total"]*100
        
        list_products = list(set(self.export_types["product_category"]))
        
        i = 0 
        for product in list_products:
            
            temp = self.export_types[self.export_types["product_category"] == product].copy()
            
            G_w=nx.from_pandas_edgelist(temp, 
                "exporter_code", "importer_code", ["product_export_fraction"], create_using = nx.DiGraph())
            centrality_product_w = nx.eigenvector_centrality(G_w, weight = "product_export_fraction", 
                                                           max_iter= 10000)

            G=nx.from_pandas_edgelist(temp,"exporter_code", "importer_code", create_using = nx.DiGraph())
            centrality_product = nx.eigenvector_centrality(G,max_iter= 10000)

            if(i == 0):
                self.centrality_product = pd.DataFrame(list(map(list, centrality_product.items())), 
                                               columns = ["country_code", "prod_" + product])
                

            else: 
                self.centrality_product = pd.merge(self.centrality_product, 
                                               pd.DataFrame(list(map(list, centrality_product.items())), 
                                               columns = ["country_code", "prod_" + product]), 
                                                  on = "country_code")
                
            self.centrality_product = pd.merge(self.centrality_product, 
                                               pd.DataFrame(list(map(list, centrality_product_w.items())), 
                                               columns = ["country_code", "prod_w_" + product]), 
                                                  on = "country_code")
            
            i = i+1         
    
    def combine_normalize_features(self):
        
        self.combined_features = pd.merge(self.features, self.centrality_overall,on = "country_code")
        self.combined_features = pd.merge(self.combined_features, self.centrality_product,on = "country_code")
        #step eliminates NA and nodes that are not in graph, since they will have NA for graph features
        self.combined_features = self.combined_features.drop(columns = ["index"])
        #filter both trade data and features data to same subset of countries
        self.combined_features = self.combined_features[\
                                self.combined_features.country_code.isin(self.trade_data.importer_code)|\
                                self.combined_features.country_code.isin(self.trade_data.exporter_code)]
        self.trade_data = self.trade_data[\
                          self.trade_data.importer_code.isin(self.combined_features.country_code)&\
                          self.trade_data.exporter_code.isin(self.combined_features.country_code)]
        
        features_to_norm = list(self.combined_features.columns.copy())
        non_norm = ["country_code", "node_numbers"]
        cols_insufficient_data = list(self.combined_features.loc[:, self.combined_features.nunique() < 2].columns.copy())
        non_norm.extend(cols_insufficient_data)
 
        features_to_norm = [x for x in features_to_norm if x not in non_norm]
        scaler = StandardScaler()
        #we preserve NAs in the scaling
        self.combined_features[features_to_norm] = scaler.fit_transform(self.combined_features[features_to_norm])
        self.combined_features.fillna(0, inplace = True) #we fill NA after scaling 
        #check that feature has at least 20% coverage in a given year -- otherwise set to NA
        for feature in self.feature_list:
            coverage = len(self.combined_features[self.combined_features[feature] != 0])/len(self.combined_features)
            if(coverage <= 0.20): self.combined_features[feature] = 0

In [ ]:
data_dict = {}

#GDP data goes up to 2018 -- trange up to 2019 spans up to 2018
for year in trange(1962, 2019):
    trade = CreateFeatures(year = year)
    trade.prepare_econ_features()
    trade.prepare_network_features()
    trade.combine_normalize_features()
    
    data_dict[year] = trade

In [ ]:
data_dict[2000].combined_features

In [ ]:
with open("../data/features_dict.pkl", "wb") as f:
    pkl.dump(data_dict, f)

In [5]:
with open("../feature_dicts/features_dict.pkl", "rb") as f:
    data_dict = pkl.load(f)

In [6]:
# Initialize variables to keep track of the DataFrame with the most rows
max_rows = 0
df_with_max_rows = None

# Iterate over the dictionary
for key, df in data_dict.items():
    # If this DataFrame has more rows than the current maximum, update the maximum and the DataFrame
    if len(df.combined_features) > max_rows:
        max_rows = len(df.combined_features)
        df_with_max_rows = df.combined_features
        max_year = key

In [7]:
# Compute the variance of each column
numeric_columns = df_with_max_rows.drop(['country_code'], axis=1)
variances = numeric_columns.var()

# Find columns with variance less than 0.1 (this is the threshold, adjust as needed)
columns_to_drop = variances[variances < 0.1].index
filtered_df = df_with_max_rows.drop(columns_to_drop, axis=1)

In [8]:
# Iterate over the dictionary
for key, df in data_dict.items():
    df.combined_features = df.combined_features.drop(columns_to_drop, axis=1)

## Mutual Information Selection

In [9]:
X_train = filtered_df.drop(['country_code','current_gdp_growth'], axis = 1)
Y_train = filtered_df['current_gdp_growth']

In [10]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectPercentile

mutual_info = mutual_info_regression(X_train, Y_train)

mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

selected_top_columns = SelectPercentile(mutual_info_regression, percentile=2)
selected_top_columns.fit(X_train, Y_train)
selected_top_columns.get_support()

array([False, False, False, ..., False, False, False])

In [11]:
columns = X_train.columns[selected_top_columns.get_support()]
X_train = X_train[columns]

In [12]:
column_list = list(columns)
column_list.append('current_gdp_growth')
column_list.append('country_code')

In [13]:
for key, df in data_dict.items():
    df.combined_features = df.combined_features[column_list]

In [14]:
columns_all_zeros = []
for key, df in data_dict.items():
    zero_columns = [col for col in df.combined_features.columns if (df.combined_features[col] == 0).all()]

    # Iterate over list1
    for item in zero_columns:
        # If the item is not in list2, add it
        if item not in columns_all_zeros:
            columns_all_zeros.append(item)


In [15]:
for key, df in data_dict.items():
    df.combined_features = df.combined_features.drop(columns_all_zeros, axis=1)

In [16]:
data_dict[1962].combined_features

,IT.MLT.MAIN.P2,NE.CON.PRVT.KD.ZG,NE.CON.TOTL.KD.ZG,NV.IND.TOTL.KD.ZG,NV.SRV.TOTL.KD.ZG,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.KD.ZG,NY.GNP.MKTP.KD.ZG,NY.GNP.PCAP.KD.ZG,SP.ADO.TFRT,SP.POP.2024.FE.5Y,SP.POP.2024.MA.5Y,SP.POP.6569.FE.5Y,SP.POP.6569.MA.5Y,SP.POP.65UP.MA.ZS,current_gdp_growth,country_code
0,-0.111774,0.000000,-1.024184,0.000000,0.000000,-1.112815,-1.038860,-0.703257,-0.630105,-0.892056,0.112493,-0.161362,0.282761,0.626574,0.204211,-1.112815,ARG
1,1.301831,-0.340188,-0.246697,0.000000,0.000000,-0.714648,-0.780316,0.000000,-0.351662,-1.104107,-1.529665,-1.375682,1.179255,1.004898,1.104184,-0.714648,AUS
2,0.097787,0.000000,0.000000,0.000000,0.000000,-0.463491,-0.153232,0.000000,0.000000,-0.882694,-0.717122,0.877274,2.809942,2.482537,2.321666,-0.463491,AUT
3,0.417067,0.000000,0.000000,0.000000,0.000000,0.011966,0.395375,0.000000,0.000000,-1.402846,-2.383251,-2.241161,2.496846,2.458060,2.431750,0.011966,BEL
4,-0.685382,-0.913001,-0.700270,0.000000,0.000000,-1.590323,-1.541320,-1.038805,-0.983784,0.733933,0.551797,0.097609,0.329067,0.341150,0.307165,-1.590323,BEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,-0.611217,0.000000,0.000000,0.000000,0.000000,0.078634,-0.011029,0.000000,0.000000,0.635345,0.119556,0.378782,-0.147779,-0.315724,-0.368509,0.078634,TUR
72,-0.129166,0.000000,0.102288,0.000000,0.000000,-1.246832,-1.099825,0.000000,0.000000,-0.759923,-0.112155,-0.163143,0.871415,1.106939,1.135215,-1.246832,URY
73,2.761470,0.000000,0.000000,0.000000,0.000000,0.176675,0.333370,0.000000,0.000000,-0.553262,-1.746300,-2.167133,1.292109,1.442964,1.559219,0.176675,USA
74,-0.495877,-0.245292,-0.328785,0.000000,0.000000,0.627946,0.396841,0.655605,0.504555,0.276462,0.416738,0.129565,-0.947292,-0.888139,-0.864482,0.627946,VEN


In [16]:
with open("../data/filtered_features_dict.pkl", "wb") as f:
    pkl.dump(data_dict, f)

## Random Subset Selection

In [9]:
X_train = filtered_df.drop(['country_code','current_gdp_growth'], axis = 1).iloc[:,1:-23]

In [10]:
random_columns = X_train.sample(n=15, axis=1).columns

In [17]:
random_columns

Index(['GC.XPN.TOTL.CN', 'DC.DAC.DNKL.CD', 'ST.INT.DPRT', 'SP.POP.6064.FE.5Y',
       'SL.IND.EMPL.MA.ZS', 'DT.NFL.PRVT.CD', 'SE.XPD.CSEC.ZS',
       'SP.URB.TOTL.IN.ZS', 'NY.GNP.MKTP.PP.KD', 'DC.DAC.POLL.CD',
       'TX.VAL.MRCH.R5.ZS', 'SE.PRM.OENR.FE.ZS', 'NE.CON.GOVT.CN',
       'SL.TLF.BASC.ZS', 'EN.ATM.PM25.MC.M3'],
      dtype='object')

In [11]:
# Iterate over the dictionary
for key, df in data_dict.items():

    # Combine the lists
    all_cols = ['country_code','current_gdp_growth'] + random_columns.tolist()

    # Select the columns from the dataframe
    df.combined_features = df.combined_features[all_cols]

In [16]:
with open("../feature_dicts/random_features_dict.pkl", "wb") as f:
    pkl.dump(data_dict, f)